Project: Design and implement an LLM powered chatbot. This chatbot will be able to have a conversation and remember interaction history.

In [170]:
# Environment Setup
import os  # Import the os module for environment variable handling
from dotenv import load_dotenv  # Import dotenv to load environment variables from .env files

load_dotenv()  # Load the environment variables from .env files

# Retrieve the Groq API key from environment variables
groq_api_key = os.getenv("GROQ_API_KEY")
if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found. Check your .env file.") # Raise an error if the key is not found
else:
    print(f"The GROQ API Key is: {groq_api_key}") # Print the API key for confirmation

The GROQ API Key is: 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


In [171]:
# Model Initialization
from langchain_groq import ChatGroq # Import the ChatGroq class from langchain_groq

# Initialize the chatbot model using the specified model name and API key
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B48819F150>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B48819C390>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [172]:
# Single Turn Interaction
from langchain_core.messages import HumanMessage # Import HumanMessage for user input

# Send a human message to the model to initiate a conversation
model.invoke([HumanMessage(content="Hi, My name is Dahab and I am a Senior AI Engineer")])


AIMessage(content="Hello Dahab, it's nice to meet you! That's impressive, a Senior AI Engineer! \n\nWhat kind of projects are you working on these days?  I'm always interested in learning more about what people are doing in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 23, 'total_tokens': 81, 'completion_time': 0.105454545, 'prompt_time': 0.00013801, 'queue_time': 0.018804977, 'total_time': 0.105592555}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-766becd0-4e10-4011-87a9-1ffe1cc503c9-0', usage_metadata={'input_tokens': 23, 'output_tokens': 58, 'total_tokens': 81})

In [173]:
# Multi-turn Interaction
from langchain_core.messages import AIMessage # Import AIMessage for responses from the model

# Send a series of messages including a conversation history
model.invoke(
    [
        HumanMessage(content="Hi, My name is Dahab and I am a Senior AI Engineer"),
        AIMessage(content="Hello Dahab, it's nice to meet you! \n\nAs a Senior AI Engineer yourself, I'm sure you have a lot of experience and knowledge to share. What kind of AI projects are you currently working on?  \n\nI'm always eager to learn more about the latest advancements in the field.\n"),
        HumanMessage(content="Hi, what's my name and what do I do?")
    ]
)

AIMessage(content="You are Dahab, and you are a Senior AI Engineer. \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  I'm curious to learn more! \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 112, 'total_tokens': 159, 'completion_time': 0.085454545, 'prompt_time': 0.003870707, 'queue_time': 0.02026097, 'total_time': 0.089325252}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5fbe82bd-732d-4089-9c45-8f09e810f2a5-0', usage_metadata={'input_tokens': 112, 'output_tokens': 47, 'total_tokens': 159})

## Message History
Use Message History to keep track of inputs and outputs of the model, store them in a datastore. Future interactions will load these messages and pass them into the chain as part of the input. 

In [174]:
# Session Management
from langchain_community.chat_message_histories import ChatMessageHistory  # Import history for chat sessions
from langchain_core.chat_history import BaseChatMessageHistory  # Base class for chat histories
from langchain_core.runnables.history import RunnableWithMessageHistory  # To handle sessions with message history

## Function to create a session ID for each chat
store={} # Dictionary to store chat session history
def get_session_history(session_id: str) -> BaseChatMessageHistory: 
    if session_id not in store: # Check if session history exists
        store[session_id] = ChatMessageHistory() # Create new history if not present
    return store[session_id] # Return the chat session history for the session ID

# Create a runnable model instance that maintains message history
with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [175]:
# First chat session with session ID 'chat1'
config={"configurable":{"session_id":"chat1"}} # Configuration dictionary for session identification

In [176]:
# Invoke the model within the first session
response=with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Dahab and I am a Senior AI Engineer")], # User prompt
    config=config # Use session configuration
)

In [177]:
response.content # Output the AI's response

"Hello Dahab, it's nice to meet you!\n\nAs a Senior AI Engineer, I imagine you have a lot of interesting projects and challenges you work on. What are some of the things you're most passionate about in the field of AI?\n"

In [178]:
# Follow-up question to check the AI's knowledge of user identity
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

AIMessage(content='Your name is Dahab.  You told me at the beginning of our conversation! 😊 \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 91, 'total_tokens': 124, 'completion_time': 0.06, 'prompt_time': 0.002592481, 'queue_time': 0.022756358, 'total_time': 0.062592481}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6afd5250-7e69-4161-84a1-038ec3bb907e-0', usage_metadata={'input_tokens': 91, 'output_tokens': 33, 'total_tokens': 124})

In [179]:
# Change the session ID configuration for a new chat session
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What is my name?")], # User question in new session
    config=config1
)
response.content

"As a large language model, I do not have access to any personal information about you, including your name. If you'd like to tell me your name, I'd be happy to know!\n"

In [180]:
# User provides a name in the new session
response=with_message_history.invoke(
    [HumanMessage(content="My name is John.")], # User provides a name
    config=config1 # Use the new session configuration
)
response.content # Output the AI's response

"Hello, John! It's nice to meet you.  Is there anything I can help you with today?\n"

In [181]:
# Check AI's understanding of the new user's name
response=with_message_history.invoke(
    [HumanMessage(content="What is my name?")], # Follow-up question
    config=config1
)
response.content

'Your name is John. 😊 \n\nI remember!  \n\nIs there anything else I can help you with, John?\n'

Prompt Tempalate: Turn raw user information into LLM usable format.  

In [182]:
# Prompt Setup for Basic Interaction
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Creating a chat prompt template with a system message and message placeholders for dynamic content
prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Remember user details and respond accordingly.", # System instruction
        ),
        MessagesPlaceholder(variable_name="messages"), # Placeholder for dynamic message content
    ]
)



In [183]:
# Model Integration
# Combine the prompt with the model to create a processing chain
chain=prompt|model
# Invoke the chain with initial user input
chain.invoke({"messages":[HumanMessage(content="Hi My name is Dahab.")]})

AIMessage(content="Hello Dahab, it's nice to meet you! I'll remember your name. \n\nIs there anything I can help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 29, 'total_tokens': 66, 'completion_time': 0.067272727, 'prompt_time': 0.00015918, 'queue_time': 0.021137617, 'total_time': 0.067431907}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6e24ae6e-b33d-4b56-a373-4f6da49049eb-0', usage_metadata={'input_tokens': 29, 'output_tokens': 37, 'total_tokens': 66})

In [184]:
# Enabling Memory with Runnable
# Initialize a runnable with message history for managing session-specific interactions
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

# Configuration for a new session with session ID 'chat3'
config = {"configurable": {"session_id": "chat3"}}

# Invoking the model within a session to maintain context
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Dahab."),],
    config=config
)
response # Display the model's response

AIMessage(content="Hello Dahab, it's nice to meet you!  \n\nI'll remember your name. How can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 29, 'total_tokens': 63, 'completion_time': 0.061818182, 'prompt_time': 0.000141959, 'queue_time': 0.020891409, 'total_time': 0.061960141}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fdf4396f-4e7f-4914-9cf5-273faae9e5a0-0', usage_metadata={'input_tokens': 29, 'output_tokens': 34, 'total_tokens': 63})

In [185]:
# Prompt Enhancement with Complex Instructions
# Import prompt components for enhanced templating
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Enhanced prompt setup with language customization
prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all the questions to the best of your ability in {language}.", # System instruction with variable language support
        ),
        MessagesPlaceholder(variable_name="messages"), # Dynamic message content
    ]
)



In [186]:
# Advanced Model Chain and Interaction
# Combine updated prompt with model to process language-specific requests
chain=prompt|model

# Invoke the chain with a language specification
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is Dahab")],"language":"Hindi"})
response.content # Display response content in specified language

'नमस्ते दाहब! मैं आपकी मदद करने के लिए तैयार हूँ। आप मुझसे कोई भी प्रश्न पूछ सकते हैं, मैं अपनी पूरी क्षमता के अनुसार उत्तर दूंगा। 😊 \n\n'

Wrap this additional chain complexity in a Message History class. 

In [187]:
# Enhanced Session Management
# Enhance the runnable to handle complex input structures
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages" # Specify the input key for message history
)

In [188]:
# Configuration for a new session with session ID 'chat4'
config = {"configurable": {"session_id": "chat4"}}

# Interactive response handling with language specificity
respose=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi, I am Dahab")],"language":"Hindi"},
    config=config
)
response.content # Display AI response

'नमस्ते दाहब! मैं आपकी मदद करने के लिए तैयार हूँ। आप मुझसे कोई भी प्रश्न पूछ सकते हैं, मैं अपनी पूरी क्षमता के अनुसार उत्तर दूंगा। 😊 \n\n'

In [189]:
# Checking AI's memory and understanding in the session
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="What's my name?")], "language": "Hindi"},
    config=config
)
response.content  # Display AI response confirming user info


'आपका नाम दाहब है।  \n'

# Managing the Conversation History
To keep the messages within bound and context window. Limit the size of messages being parsed. 

In [224]:
# Message Trimming Setup
from langchain_core.messages import SystemMessage, trim_messages

# Create a trimmer to control message size and inclusion criteria
trimmer = trim_messages(
    max_tokens=45,  # Limit the message length to a maximum of 45 tokens
    strategy="last",  # Use the strategy to keep the last relevant messages within the limit
    token_counter=model,  # Use model to count tokens
    include_system=True,  # Include system messages in the trimming process
    allow_partial=False,  # Do not allow partial messages
    start_on="human"  # Begin trimming from the first human message
)

# Define a series of initial messages in the conversation
messages = [
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="Hi! I'm Dan"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like Chicken Sandwiches"),
    AIMessage(content="That's nice"),
    HumanMessage(content="What is 2+2?"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks!"),
    AIMessage(content="No problem!"),
    HumanMessage(content="Having fun?"),
    AIMessage(content="Yes!"),
]

# Apply the trimmer to the messages
trimmer.invoke(messages)

[SystemMessage(content='You are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 2+2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})]

In [225]:
# Chain Creation and Initial Interaction
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

# Define a processing chain that trims and passes messages
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer) # Chain to process and trim messages
    | prompt
    | model
)

# Invoke the chain with a query about preferences
response=chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What sandwich do I like?")], # Add user question to the message
        "language": "English"
    }

)
response.content

"As an AI, I don't have access to your personal information or preferences, so I wouldn't know what kind of sandwich you like! 🥪\n\nWhat's your favorite sandwich? 😊  \n"

In [231]:
# Memory Enhancement and Problem Inquiry
# Invoke the chain with a follow-up question about previous interactions
response=chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What match problem did I ask?")], # User asks about previous math question
        "language": "English"
    }

)
response.content # Output AI response about previous questions

'You asked "What is 2+2?"  \n\n\n\nLet me know if you want to try another one! 😊 \n'

In [228]:
# New Interactive Session 
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history, # Function to manage and retrieve session history
    input_messages_key="messages" # Specify the input key for message history
)

# Set the configuration for session tracking
config = {"configurable": {"session_id": "chat5"}}

In [229]:
# Handle interactive response with no session history - Question 1
respose=with_message_history.invoke(
    {
        'messages': messages + [HumanMessage(content="What is my name?")], # User asks a follow-up question
        "language": "English" # Maintain English context
    },
    config=config, # Apply session configuration
)
response.content # Display AI response based on session memory

"As an AI, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  \n"

In [230]:
# Handle interactive response with no session history - Question 2
respose=with_message_history.invoke(
    {
        'messages': [HumanMessage(content="What math problem did I ask?")], # User asks a follow-up question
        "language": "English" # Maintain English context
    },
    config=config, # Apply session configuration
)
response.content # Display AI response based on session memory

"As an AI, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  \n"